# Multi-Modality Fusion Model


### INDEX:

**1)  [Mount G-Drive](#scrollTo=Jm42Bj4po3Y-)**<br/>
**2)  [VGG-16](#scrollTo=kOje8AUqxGz9)**<br/>
**3)  [ResNet-50](#scrollTo=yoaXkLst0vxa)**<br/>
**4)  [EffecientNet-V3](#scrollTo=9CGehUvs0u_p)**<br/>
**5)  [Fusion](#scrollTo=Qfl5gZAn23fO)**<br/>


------

### Header files

In [1]:
# our utils file
import utils
from models import VGG, ResNet, EffNetB3

# basic ML
import os
import numpy as np
import matplotlib.pyplot as plt

# connect to drive
from google.colab import drive

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm

### Mounting G-Drive

In [2]:
drive.mount('/content/gdrive')
pkldest = '/content/gdrive/MyDrive/pkl_files/'
dataset = '/content/gdrive/MyDrive/Minidata'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


------
## VGG 16

<img src="https://user-images.githubusercontent.com/66634743/113575262-a8d31580-962e-11eb-840a-83f18c2029e3.png" height=150 width=600>


### Loading data from pkl

In [3]:
images = utils.load_from_pkl(pkldest+"vgg16_images.pkl")
scenes = utils.load_from_pkl(pkldest+"scenes.pkl")

print("Shape of Images = ", images.shape)
print("Shape of Scenes = ", scenes.shape)

Shape of Images =  (5059, 224, 224, 3)
Shape of Scenes =  (5059,)


### Building the model

In [4]:
vgg = VGG()
vgg.freeze()

vgg_features = vgg.predict(images=images)

159/159 [==============================] - 19s 61ms/step


### Dumping data into .pkl files

In [5]:
if("vgg16_features.pkl" not in os.listdir(pkldest)):
    utils.dump_into_pkl(vgg_features, pkldest+"vgg16_features.pkl")

### PCA (k = 512)

In [6]:
scenes = utils.prep_labels(scenes)
k = 512

pca = PCA(n_components=k)

pca.fit(vgg_features)
PCA_features = pca.transform(vgg_features)

print("Shape after PCA ", PCA_features.shape)

Shape after PCA  (5059, 512)


### Classifier

In [7]:
X_train, X_test, y_train, y_test = train_test_split(PCA_features, scenes, test_size=0.2, random_state=42)

classifier = svm.SVC(kernel='rbf', probability=True)

classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### Temporary Results

In [8]:
pred = classifier.predict(X_test)

acc = np.sum(1*(pred==y_test))/y_test.shape[0]
print("VGG + SVM  = ", acc*100)

VGG + SVM  =  84.38735177865613


### Prababilities of classes

In [9]:
vgg16_probs = classifier.predict_proba(X_test)

print(vgg16_probs.shape)

(1012, 10)


------
## ResNet50

<img src="https://user-images.githubusercontent.com/66634743/113575413-dfa92b80-962e-11eb-902c-380e39b91234.png" height=150 width=600>

### Loading data from pkl



In [10]:
images = utils.load_from_pkl(pkldest+"resnet_images.pkl")
scenes = utils.load_from_pkl(pkldest+"scenes.pkl")

print("Shape of Images = ", images.shape)
print("Shape of Scenes = ", scenes.shape)

Shape of Images =  (5059, 512, 512, 3)
Shape of Scenes =  (5059,)


### Building the model

In [11]:
resnet = ResNet()
resnet.freeze()

resnet_features = resnet.predict(images=images)

159/159 [==============================] - 37s 223ms/step


### Dumping data into .pkl files

In [12]:
if("resnet_features.pkl" not in os.listdir(pkldest)):
    utils.dump_into_pkl(resnet_features, pkldest+"resnet_features.pkl")

### PCA (k = 128)

In [13]:
scenes = utils.prep_labels(scenes)
k = 128

pca = PCA(n_components=k)

pca.fit(resnet_features)
PCA_features = pca.transform(resnet_features)

print("Shape after PCA ", PCA_features.shape)

Shape after PCA  (5059, 128)


### Classifier

In [14]:
X_train, X_test, y_train, y_test = train_test_split(PCA_features, scenes, test_size=0.2, random_state=42)

classifier = svm.SVC(kernel='rbf', probability=True)

classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### Temporary Results

In [15]:
pred = classifier.predict(X_test)

acc = np.sum(1*(pred==y_test))/y_test.shape[0]
print("ResNet50 + SVM  = ", acc*100)

ResNet50 + SVM  =  90.11857707509881


### Prababilities of classes

In [16]:
resnet_probs = classifier.predict_proba(X_test)

print(resnet_probs.shape)

(1012, 10)


------
## EfficientNetB3

<img src="https://user-images.githubusercontent.com/66634743/113626999-ec03a780-9673-11eb-9779-21d376f419d2.png" height=150 width=600>

### Loading data from pkl


In [17]:
images = utils.load_from_pkl(pkldest+"Effnetb3_images.pkl")
scenes = utils.load_from_pkl(pkldest+"scenes.pkl")

print("Shape of Images = ", images.shape)
print("Shape of Scenes = ", scenes.shape)

Shape of Images =  (5059, 300, 300, 3)
Shape of Scenes =  (5059,)


### Building the model

In [18]:
effnetB3 = EffNetB3()

effnetB3.freeze()

effnetB3_features = effnetB3.predict(images=images)

159/159 [==============================] - 20s 110ms/step


### Dumping data into .pkl files

In [19]:
if("effnetB3_features.pkl" not in os.listdir(pkldest)):
    utils.dump_into_pkl(effnetB3_features, pkldest+"effnetB3_features.pkl")

### PCA (k = 128)

In [20]:
scenes = utils.prep_labels(scenes)
k = 128

pca = PCA(n_components=k)

pca.fit(effnetB3_features)
PCA_features = pca.transform(effnetB3_features)

print("Shape after PCA ", PCA_features.shape)

Shape after PCA  (5059, 128)


### Classifier

In [21]:
X_train, X_test, y_train, y_test = train_test_split(PCA_features, scenes, test_size=0.2, random_state=42)

classifier = svm.SVC(kernel='rbf', probability=True)

classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### Temporary Results

In [22]:
pred = classifier.predict(X_test)

acc = np.sum(1*(pred==y_test))/y_test.shape[0]
print("EffNetb3 + SVM  = ", acc*100)

EffNetb3 + SVM  =  90.11857707509881


### Prababilities of classes

In [23]:
effnet_probs = classifier.predict_proba(X_test)

print(effnet_probs.shape)

(1012, 10)


------

## Fusion

<img src="https://user-images.githubusercontent.com/66634743/113685129-f99e4900-96d6-11eb-985b-e477cdc3074b.png" height=400 width=800>

### Fusing the predictions of the 3 models

In [24]:
merged = (effnet_probs + resnet_probs + vgg16_probs)/3

merged_preds = np.argmax(merged, axis=1)

### Final Multi-Modality Model Result

In [25]:
final_acc = np.sum(y_test == merged_preds)/merged_preds.shape[0]

print("Multi-Modality Model : ", final_acc)

Multi-Modality Model :  0.9140316205533597
